# FAIR and scalable management of small-angle X-ray scattering data  
## Module 1: Conversion of PDH to AnIML

> Authors: Torsten Giess, Selina Itzigehl, Jan Range, Johanna Bruckner, Juergen Pleiss  
> Last modified: 23.02.2022

---

### **Abstract** <a class="anchor" name="abstract"></a>

Using novel Python package [pyAnIML](https://github.com/FAIRChemistry/pyAnIML) (version 0.0.2), established packages [lxml](https://lxml.de/) (version 4.8.0) and [pandas](https://pandas.pydata.org/) (version 1.4.1), as well as various packages from the Python 3 standard libary, this notebook provides the means to create new [AnIML](https://animl.org/) documents and add entries to existing ones. 

---

### **Table of Contents** <a class="anchor" name="table_of_contents"></a>

- [Abstract](#abstract)
- [Workflow](#workflow)
    - [User guide](#user_guide)
    - [Preparation](#preparation)
    - [Conversion to AnIML](#conversion)
    - [Adding to an an existing AnIML document](#addition)
    - [Inspecting an AnIML document](#inspection)
- [Disclosure](#disclosure)

---

### **Workflow** <a class="anchor" name="workflow"></a>

Following is the workflow for Module 1 of FAIR and scalable management of small-angle X-ray scattering data.

#### **User guide** <a class="anchor" name="user_guide"></a>

This notebook can be used to either build up a [new AnIML object](#conversion) in memory and later serialize it to an actual document on disk or to extend an already [existing AnIML document](#addition) on disk by adding new entries to it and reserializing it later.  
In both cases the required fields and explanations for them are already present in this notebook. As such, it can be used as is or extended if needed.

#### **Preparation** <a class="anchor" name="preparation"></a>

This section contains the necessary preparations for using this module. Code cells in this section are required regardless of which functionality of this notebook is used. First, the required packages from the [Python 3 standard library](https://docs.python.org/3/library/), the Python Package Index ([PyPI](https://pypi.org/)), and *ad hoc* modules of this work are imported. Then, both current time and path are retrieved and stored in the desired formats.

In [ ]:
print("Importing standard library packages.")
from datetime import date
from pathlib import Path
print ("Done.")

In [ ]:
print("Importing PyPI packages.")
from pyaniml import AnIMLDocument, Sample, Series, ExperimentStep, Device, IndividualValueSet, SeriesSet, Category
from pyaniml.core.method import Author, Software
print ("Done.")

In [ ]:
print("Importing local packages.")
from modules.pdhreader import PDHReader
print("All done.")

In [ ]:
date_suffix = str(date.today()).replace("-", "")[2:]

In [ ]:
cwd = Path.cwd()
path_to_datasets = cwd / "./datasets/"

#### **Conversion of PDH to AnIML** <a class="anchor" name="conversion"></a>

With this section of the notebook, a new AnIML document can be created in memory and later be serialized to XML. The presented code cells provide the most important API calles for building an AnIML document with pyAnIML.  
Running a code cell more than once will add the contents of this cell to the AnIML document accordingly more than once. This can be used to add multiple samples, experiments, series, ... to the same document by simple changing the respective code cell before serializing it.

1. Create a new AnIML document as an object in memory or access an existing document:

- Case a) Create a new object:

In [ ]:
animl_doc = AnIMLDocument()

- Case b) Access an existing document:

In [122]:
path_to_AnIML_file = path_to_datasets / f"processed/fairsaxs_220320.animl"
with path_to_AnIML_file.open("r") as f:
    xml_string = f.read()
    animl_doc = AnIMLDocument.fromXMLString(xml_string)

2. Call the PDH reader with the path to the directory holding the PDH files as argument, retrieve a list of available files, and show this list:

In [124]:
pdh_dir = PDHReader(path_to_datasets / "raw/OTAC_measurement_data/OTAC_000wtp_T025")
dict_of_files = pdh_dir.available_files()
for index, file in dict_of_files.items():
    print(f"{index}: {file}")

23:29:24 - modules.pdhreader - DEBUG: Constructor called, 'PDHReader'@0x1cdfa021750 initialised.
23:29:24 - modules.pdhreader - DEBUG: Destructor called, 'PDHReader'@0x1cdfa0204c0 deleted.
0: OTAC_001wtp_T025[5]
1: OTAC_005wtp_T025[5]
2: OTAC_010wtp_T025[5]
3: OTAC_020wtp_T025[5]
4: OTAC_030wtp_T025[5]
5: OTAC_040wtp_T025[5]
6: OTAC_050wtp_T025[5]
7: OTAC_060wtp_T025[5]
8: OTAC_061wtp_T025[5]
9: OTAC_062wtp_T025[5]
10: OTAC_063wtp_T025[5]
11: OTAC_064wtp_T025[5]
12: OTAC_065wtp_T025[5]
13: OTAC_066wtp_T025[5]
14: OTAC_067wtp_T025[5]
15: OTAC_068wtp_T025[5]
16: OTAC_069wtp_T025[5]
17: OTAC_070wtp_T025[5]
18: OTAC_080wtp_T025[5]
19: OTAC_090wtp_T025[5]
20: OTAC_091wtp_T025[5]
21: OTAC_092wtp_T025[5]
22: OTAC_093wtp_T025[5]
23: OTAC_094wtp_T025[5]
24: OTAC_095wtp_T025[5]
25: OTAC_100wtp_T025[5]


3. Select the desired file either by name or by list index and extract the data as pandas dataframe:

In [125]:
pdh_file = dict_of_files[25]
raw_dataframe = pdh_dir.extract_data(pdh_file)
print(raw_dataframe)

23:29:32 - modules.pdhreader - DEBUG: Data extracted from 'OTAC_100wtp_T025[5]'.
      scattering_vector  counts_per_area
0              0.114488     3.016519e-10
1              0.121128     1.856291e-09
2              0.127769     2.245582e-09
3              0.134409     1.933379e-11
4              0.141050     1.440021e-10
...                 ...              ...
1092           7.284063     2.263677e-02
1093           7.290468     1.354665e-02
1094           7.296871     1.313607e-02
1095           7.303275     2.721327e-02
1096           7.309678     2.634353e-02

[1097 rows x 2 columns]


4. Start building up the AnIML document. Create a new sample with an ID and name and add it to the AnIML object:

In [126]:
experiment_name = f"{pdh_file[:4]}/water: x = {pdh_file[5:8]} wt%; T = {pdh_file[-5:-3]} C"
print(experiment_name)

OTAC/water: x = 100 wt%; T = 25 C


In [127]:
new_sample = Sample(
    id=pdh_file.replace("%", "p")[:-3],
    name=experiment_name
)
print(new_sample)

Sample(name='OTAC/water: x = 100 wt%; T = 25 C', id='OTAC_100wtp_T025', properties=[])


In [128]:
animl_doc.add_sample(new_sample)

5. Create or access an experiment step for the AnIML object, providing it with a name and an ID:

- Case a) Create a new experiment step object:

In [129]:
experiment_step = ExperimentStep(
    name=f"Sample data for {experiment_name}",
    experiment_step_id=pdh_file.replace("%", "p")[:-3]
)
print(experiment_step)

ExperimentStep(name='Sample data for OTAC/water: x = 100 wt%; T = 25 C', experiment_step_id='OTAC_100wtp_T025', infrastructure=Infrastructure(sample_references=SampleReferenceSet(sample_references=[])), method=Method(methods=[]), result=Result(results=[]))


- Case b) Access an existing experiment step within an AnIML document:

In [ ]:
available_experiment_steps = experiment_step = animl_doc.experiment_step_set.experiment_steps
print([step.name for step in available_experiment_steps])

In [ ]:
experiment_step = available_experiment_steps[0]

6. Create one or more sample references to samples that were part of this experiment step, providing a sample object (e.g. new_sample), a role, and a purpose:

In [130]:
experiment_step.add_sample_reference(
    sample=new_sample,
    role="sample",
    sample_purpose="consumed"
)

7. If applicable, create the device and/or software on which the sample(s) were measured/analysed and add them to the experiment step object:

In [131]:
new_author = Author(
    user_type="human",
    name="Selina Itzigehl",
    affiliation="University of Stuttgart",
    email="selina.itzigehl@ipc.uni-stuttgart.de",
    role="",
    phone="",
    location=""
)
experiment_step.add_method(new_author)

In [132]:
new_author = Author(
    user_type="human",
    name="Johanna Bruckner",
    affiliation="University of Stuttgart",
    email="johanna.bruckner@ipc.uni-stuttgart.de",
    role="",
    phone="",
    location=""
)
experiment_step.add_method(new_author)

In [133]:
new_device = Device(
    manufacturer="Anton Parr",
    name="SAXSess",
    device_id="",
    firmware_version="",
    serial_number=""
)
experiment_step.add_method(new_device)

In [134]:
new_software = Software(
    manufacturer="Project Jupyter",
    name="JupyterLab",
    version="3.2.9",
    operating_system="Microsoft Windows 10 Pro 10.0.19044 Build 19044"
)
experiment_step.add_method(new_software)

In [135]:
new_software = Software(
    manufacturer="OriginLab",
    name="OriginPro 2021b (64-bit) SR2",
    version="9.8.5.212",
    operating_system="Microsoft Windows 10 Pro 10.0.19044 Build 19044"
)
experiment_step.add_method(new_software)

8. Create a series for every dimension of data present in the dataframe. The data from the dataframe goes into an IndividualValueSet in form of a list, which is then added to the actual series object, together with a name, ID, the data type, the dependency, and plot scale:

- Case a) Create a new category object:

In [136]:
category = Category(name="Measurements")

- Case b) Access an existing category:

In [ ]:
available_categories = experiment_step.result.results
print([content.name for content in available_categories])

In [ ]:
category = available_categories[0]

- Handle series:

In [137]:
q_values = IndividualValueSet(
    raw_dataframe["scattering_vector"].tolist()
)
q = Series(
    name="q",
    id=f"{pdh_file.replace('%', 'p')[:-3]}_q",
    individual_value_set=q_values,
    data_type="float32",
    dependency="dependent",
    plot_scale="linear"
)
i_values = IndividualValueSet(
    raw_dataframe["counts_per_area"].tolist()
)
i = Series(
    name="I",
    id=f"{pdh_file.replace('%', 'p')[:-3]}_i",
    individual_value_set=i_values,
    data_type="float32",
    dependency="dependent",
    plot_scale="linear"
)

9. Create one or more sets for series belonging together, provide the set with a name, add it to the category object, and add it to the experiment step object:

In [138]:
new_set = SeriesSet(
    name=f"SAXS measurement",
    series=[q, i]
)

In [139]:
category.add_content(new_set)

In [140]:
experiment_step.add_result(category)

10. Finally, add the now fully built experiment step object to the AnIML object:

In [141]:
animl_doc.add_experiment_step(experiment_step)

11. Create the XML-formatted string from the AnIML object and provide path and desired file name for the actual AnIML document:

In [142]:
xml_string = animl_doc.toXML()

In [143]:
path_to_AnIML_file = path_to_datasets / f"processed/fairsaxs_220320.animl"

12. Serialize the XML string to the pathlib Path provided:

In [144]:
with path_to_AnIML_file.open("w") as f:
    f.write(xml_string)
del animl_doc, xml_string
print(f"Successfully created AnIML document.")

Successfully created AnIML document.


#### **Adding to an existing AnIML document** <a class="anchor" name="addition"></a>

An existing AnIML document can be converted into an AnIML object in memory and manipulated using the pyAnIML API. This notebook provides an example for a workflow on how to add a sample that has been forgotten to an AnIML document.

1. Give path to AnIML document to be extended in form of a pathlib Path:

In [ ]:
path_to_AnIML_file = path_to_datasets / f"processed/test/fairsaxs_220316.animl"

2. Read document as string and create AnIML object from it:

In [ ]:
with path_to_AnIML_file.open("r") as f:
    xml_string = f.read()
    animl_doc = AnIMLDocument.fromXMLString(xml_string)

3. Create the sample object:

In [ ]:
new_sample = Sample(
    id="forgotten",
    name="Forgotten sample"
)
animl_doc.add_sample(new_sample)

4. Create XML-formatted string from the AnIML object:

In [ ]:
xml_string = animl_doc.toXML()

5. Serialize the XML string to the pathlib Path provided before:

In [ ]:
with path_to_AnIML_file.open("w") as f:
    f.write(xml_string)
del animl_doc, xml_string
print(f"Successfully created AnIML document.")

#### **Inspecting an AnIML document** <a class="anchor" name="inspection"></a>

The following section is entirely optional and can be used to check if the AnIML document was serialized correctly.

1. Give path to the AnIML file as pathlib Path:

In [145]:
path_to_AnIML_file = path_to_datasets / f"processed/fairsaxs_220320.animl"

2. Open the AnIML document, create XML string, print it and delete again:

In [146]:
with path_to_AnIML_file.open("r") as f:
    control_string = f.read()
print(control_string)
del control_string

<?xml version="1.0" encoding="UTF-8"?>
<AnIML>
  <SampleSet>
    <Sample name="Cholesteryl palmitate" sampleID="CholPal_20220214"/>
    <Sample name="OTAB/water: x = 78 wt%; T = 60 C" sampleID="OTAB_078wtp_T060"/>
    <Sample name="OTAB/water: x = 82 wt%; T = 25 C" sampleID="OTAB_082wtp_T025"/>
    <Sample name="OTAB/water: x = 78 wt%; T = 58 C" sampleID="OTAB_078wtp_T058"/>
    <Sample name="OTAB/water: x = 93 wt%; T = 25 C" sampleID="OTAB_093wtp_T025"/>
    <Sample name="OTAB/water: x = 100 wt%; T = 95 C" sampleID="OTAB_100wtp_T095"/>
    <Sample name="OTAC/water: x = 050 wt%; T = 25 C" sampleID="OTAC_050wtp_T025"/>
    <Sample name="OTAC/water: x = 085 wt%; T = 74 C" sampleID="OTAC_085wtp_T074"/>
    <Sample name="OTAC/water: x = 070 wt%; T = 25 C" sampleID="OTAC_070wtp_T025"/>
    <Sample name="OTAC/water: x = 090 wt%; T = 10 C" sampleID="OTAC_090wtp_T010"/>
    <Sample name="OTAC/water: x = 094 wt%; T = 50 C" sampleID="OTAC_094wtp_T050"/>
    <Sample name="OTAC/water: x = 097 wt%;

---

### **Disclosure** <a class="anchor" name="disclosure"></a>

**Contributions**

If you wish to contribute to the F.A.I.R. Chemistry project, find us on [GitHub](https://github.com/FAIRChemistry)!

**MIT License**

Copyright (c) 2022 F.A.I.R. Chemistry

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.